In [1]:
%cd /home/teresakim/fungiclef-2024/

/home/teresakim/fungiclef-2024


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pyspark.sql.functions as f
import pyspark.ml as ml
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, IntegerType
import fungiclef.transforms as trans
from fungiclef.utils import get_spark

In [4]:
spark = get_spark(**{
    "spark.sql.parquet.enableVectorizedReader": False, 
})
display(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/19 06:13:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/19 06:13:07 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [5]:
# Get list of stored filed in cloud bucket
root = "gs://dsgt-clef-fungiclef-2024"
! date
! gcloud storage du --readable-sizes {root}/data/parquet/DF20


Fri Apr 19 06:13:46 UTC 2024


0B           gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/
0B           gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/_SUCCESS
576.12MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00000-cc0df3a9-c0ff-4f41-a2cb-442f4aae0f0c-c000.snappy.parquet
538.03MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00001-cc0df3a9-c0ff-4f41-a2cb-442f4aae0f0c-c000.snappy.parquet
534.57MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00002-cc0df3a9-c0ff-4f41-a2cb-442f4aae0f0c-c000.snappy.parquet
577.75MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00003-cc0df3a9-c0ff-4f41-a2cb-442f4aae0f0c-c000.snappy.parquet
572.16MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00004-cc0df3a9-c0ff-4f41-a2cb-442f4aae0f0c-c000.snappy.parquet
556.37MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00005-cc0df3a9-c0ff-4f41-a2cb-442f4aae0f0c-c000.snappy.parquet
589.96MiB    gs://dsgt-clef-fungiclef-2024/data/parquet/DF20/part-00006-cc0df3a9-c0ff-4f41-a2c

In [6]:

# gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/dev/train/"
# input_folder = f"dev_train.parquet"

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/"
input_folder = f"DF20"

dev_df = spark.read.parquet(gcs_parquet_path+input_folder)
dev_df.printSchema()
dev_df.count()

root
 |-- image_path: string (nullable = true)
 |-- data: binary (nullable = true)
 |-- observationID: long (nullable = true)
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- day: double (nullable = true)
 |-- countryCode: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- taxonID: double (nullable = true)
 |-- scientificName: string (nullable = true)
 |-- kingdom: string (nullable = true)
 |-- phylum: string (nullable = true)
 |-- class: string (nullable = true)
 |-- order: string (nullable = true)
 |-- family: string (nullable = true)
 |-- genus: string (nullable = true)
 |-- specificEpithet: string (nullable = true)
 |-- taxonRank: string (nullable = true)
 |-- species: string (nullable = true)
 |-- level0Gid: string (nullable = true)
 |-- level0Name: string (nullable = true)
 |-- level1Gid: string (nullable = true)
 |-- level1Name: string (nullable = true)
 |-- level2Gid: string (nullable = true)
 |-- level2Name: string (nullable = tr

295938

In [7]:
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch
import io

# load pretrained model
model = models.resnet18(pretrained=True).eval()

/home/teresakim/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/teresakim/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# transformation for image preprocessing
transform = transforms.Compose([
    transforms.Resize(256), # 256x256 px
    transforms.CenterCrop(224), # going with common crop size, haven't messed with it
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
# UDF to convert binary image data to embedding
def binary_image_to_embedding(image_data):
    img = Image.open(io.BytesIO(image_data))
    # ensure the image is in RGB format
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    with torch.no_grad():
        embedding = model(batch_t)
   
    return embedding.cpu().numpy().flatten().tolist()

In [10]:
# register UDF with the appropriate return type
binary_to_embedding_udf = udf(binary_image_to_embedding, ArrayType(FloatType()))

In [11]:
# apply UDF
df_with_embeddings = dev_df.withColumn("embeddings", binary_to_embedding_udf(dev_df["data"]))

In [12]:
df_with_embeddings.show()

gcs_embedding_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px_resnet18/"

df_with_embeddings.write.mode('overwrite').parquet(gcs_embedding_path)

24/04/19 06:15:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+-------------+------+-----+----+-----------+--------------------+-------+--------------------+-------+-------------+---------------+---------------+-----------------+--------------+---------------+---------+--------------------+---------+----------+---------+-----------+----------+--------------+-----------------+--------------------+-------------------+------------------+---------+----------+--------------------+--------+-------------+---------+--------------------+
|          image_path|                data|observationID|  year|month| day|countryCode|            locality|taxonID|      scientificName|kingdom|       phylum|          class|          order|           family|         genus|specificEpithet|taxonRank|             species|level0Gid|level0Name|level1Gid| level1Name| level2Gid|    level2Name|    ImageUniqueID|           Substrate|       rightsHolder|          Latitude|Longitude|CoorUncert|             Habitat|class_id|MetaSubstrate|po

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
